In [1]:
import pandas as pd
import glob
pd.set_option("display.max_rows", 30)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_colwidth", 1000)
pd.set_option("display.width", 2000)

In [2]:
import gensim

In [3]:
import numpy as np
np.set_printoptions(precision=4,suppress=True)
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import spacy

# PARAMETERS

In [5]:
N_TOPICS = 20
N_PASSES = 30

# READ DATA

In [71]:
frames = []
for f in glob.glob('../data/*2*.csv'):
    frames.append(pd.read_csv(f))

In [72]:
raw_df = pd.concat(frames)

In [73]:
raw_df.query('(kind==1 or kind==3) and SichterName').Content.head(5)

4                                                                                                                                                                                                                                                                             Ganz große Enttäuschung beim Anbieterwechsel ! ! ! Seit 28.01 . ohne Festnetz und Internet und keiner weiß was Sache ist . Kann ich definitiv nur von abraten . Auch der Kundenservice lässt für ein Megakonzern , wie ihr es seid , leider sehr zu Wünschen übrig . Da verspricht man sich einiges mehr ! Wut , Enttäuschung und absolute Unzufriedenheit im Umgang mit Neukunden .
288                                                                                                                                                                                                                                                                                                                                                                  

In [74]:
raw_df

,Authorid,Content,Datetime,Host,Id,IsInitialEntry,IsPrivate,SichterGruppe,SichterName,Tag,TopicId,TopicName,TwistId,Url,entryid,kind
0,2498278.0,@Telekom_hilft er kam einfach nicht ? ! Stark !,16.03.2016 08:00:08,twitter.com,24468419,True,False,NaN,NaN,abgelegt durch Sichter,551631,Alle,110c312b-58b5-4037-bbb4-00037cdb8713,https://twitter.com/Cry_Wolf_2007/status/709997664516116480#aid=117037823,e7fe91b7-17f0-41fd-b2bb-e01fb957fc77,1
1,6149628.0,Wann gibt es bei euch auch eine Allnetflat mit LTE im Prepaid Tarif ? Bei Vodafone ist das zb schon längere Zeit möglich !,28.04.2016 19:29:46,www.facebook.com,25766437,True,False,NaN,NaN,NaN,1,Client 388,e40155d8-2470-4c01-873a-0006178b3f34,https://www.facebook.com/101768433554/posts/10154087846023555#pid=101768433554&uid=101768433554_10154087846023555&aid=939071216138919&vid=2&wid=101768433554,2ab09bd6-3c6a-479f-bc8b-63540ace690c,1
2,2457147.0,Da wird fast 2W . lang an der Straße gearbeitet für schnelleres Internet und jetzt ists langsamer als vorher ! Ich glaub ich spinne ! #Telekom,04.03.2016 19:34:49,twitter.com,24072504,True,False,NaN,NaN,A-Team Twitter:Reaktion nein,0,NaN,e5ad9e6f-24b7-4b81-8a4c-000a94622b85,http://twitter.com/xLoveRemix/statuses/705823829608767488,35cee5da-c1f1-4673-931e-93c214020124,2
3,514748.0,"Ganz große Enttäuschung beim Anbieterwechsel ! ! ! Seit 28.01 . ohne Festnetz und Internet und keiner weiß was Sache ist . Kann ich definitiv nur von abraten . Auch der Kundenservice lässt für ein Megakonzern , wie ihr es seid , leider sehr zu Wünschen übrig . Da verspricht man sich einiges mehr ! Wut , Enttäuschung und absolute Unzufriedenheit im Umgang mit Neukunden .",07.04.2016 16:39:23,www.facebook.com,25163413,True,False,NaN,NaN,Prio-Fall,1,Client 388,ccc369a0-7b71-4a26-a687-000b1792564f,https://www.facebook.com/101768433554/posts/10154036549798555#pid=101768433554&uid=101768433554_10154036549798555&aid=10206765699968926&vid=2&wid=101768433554,1ed81b1b-bb5c-4a5d-9bd6-cfc3826586c5,1
4,514748.0,"Ganz große Enttäuschung beim Anbieterwechsel ! ! ! Seit 28.01 . ohne Festnetz und Internet und keiner weiß was Sache ist . Kann ich definitiv nur von abraten . Auch der Kundenservice lässt für ein Megakonzern , wie ihr es seid , leider sehr zu Wünschen übrig . Da verspricht man sich einiges mehr ! Wut , Enttäuschung und absolute Unzufriedenheit im Umgang mit Neukunden .",07.04.2016 16:39:23,www.facebook.com,25163413,True,False,Sichter FB/TW,Wechsler,Prio-Fall,1,Client 388,ccc369a0-7b71-4a26-a687-000b1792564f,https://www.facebook.com/101768433554/posts/10154036549798555#pid=101768433554&uid=101768433554_10154036549798555&aid=10206765699968926&vid=2&wid=101768433554,1ed81b1b-bb5c-4a5d-9bd6-cfc3826586c5,1
5,1577525.0,"Hallo Meli Mel , das hört sich schrecklich an . So stellen wir uns einen Wechsel nicht vor . Tut mir auch leid , dass so lange keine Lösung vorliegt . Evtl . kann ich helfen ? Wären Sie bitte so nett mir Ihre Rückrufnummer mitzuteilen ? Gerne per Nachricht bit.ly/DMBlogPK dann rufe ich Sie sobald wie möglich an . Freundliche Grüße , Felix F . external.xx.fbcdn.net/safe_image.php?d=AQCZjY9Pb8YfRYY7&w=493&h=493&url=https%3A%2F%2Ftelekomhilft.telekom.de%2Ft5%2Fimage%2Fserverpage%2Fimage-id%2F01234567890iAA0896FF1136A6A2%3Fv%3Dmpbl-1&cfs=1&sx=520&sy=0&sw=493&sh=493 https://external.xx.fbcdn.net/safe_image.php?d=AQCU6EWudyJNjp12&w=493&h=493&url=https%3A%2F%2Ftelekomhilft.telekom.de%2Ft5%2Fimage%2Fserverpage%2Fimage-id%2F01234567890iAA0896FF1136A6A2%2Fimage-size%2Foriginal%3Fv%3Dlz-1%26px%3D-1&cfs=1&sx=520&sy=0&sw=493&sh=493",14.04.2016 20:22:00,www.facebook.com,25368047,False,False,NaN,NaN,NaN,1,Client 388,ccc369a0-7b71-4a26-a687-000b1792564f,https://www.facebook.com/101768433554/posts/10154036549798555#pid=101768433554_10154036549798555&uid=10154036549798555_10154054879973555&cpid=&aid=101768433554&vid=2&wid=101768433554,f4ad8c8c-34bd-401c-bc34-dee318b50b3a,3
6,514748.0,"Kommt leider zu spät . Nach mindestens 10 solcher Nachrichten und Aussagen beim Kundenservice , 

In [75]:
pd.unique(raw_df.SichterName)

array([nan, 'Wechsler', 'GK', 'Entertain', 'IP & FTTH & PSTN', 'Festnetz',
       'Dienste & Abos', 'Neue Themen', 'Mobile'], dtype=object)

In [76]:
from collections import defaultdict

def f(x):
     return pd.Series(dict(SichterName = set(x.SichterName) - {np.nan},
                                   Tag = set(x.Tag),
                               Content = list(x.Content),
                                  Kind = list(x.kind)))

df = raw_df.query('(kind==1 or kind==3) and SichterName').sort_values(['Datetime'],ascending=True).groupby('TwistId').apply(f)

In [77]:
df

,Content,Kind,SichterName,Tag
TwistId,,,,
00090c82-7bd0-48e7-b688-28d148259a06,"[... so liebe Telekom , nach 12 Tagen wurde mein Telefonanschluss immer noch nicht aktiviert , eine ganz schön lange Zeit . Scheint schwierig zu sein sich zeitnah um das eigentliche Kerngeschäft zu kümmern . Was machen denn die vielen Telekom Mitarbeiter den ganzen Tag ? Liebe Kunden , ja mein Geschäft gibt es noch , allerdings mangels Fachkompetenz der Telekom leider ohne Telefon und Fax . So etwas ist möglich hier in Deutschland ! Würde ich in Afrika im tiefsten Busch leben , hätte ich ja Verständnis und könnte eine Buschtrommel benutzen . Sollten Rauchzeichen über unserem Ort aufgehen , diese sind von mir ... Freundliche Grüße Eva-Maria Gebels]",[1],{GK},{nan}
00e08556-0ffa-4acd-bae7-d8b0f7a5307f,"[Hat hier schon mal jemand von der Telekom weitergeholfen bekommen ? ? ? Ich glaube , das interessiert hier auch keinen von dem Verein . Schade , aber bezahlt mal einer die Rechnung nicht ... Da sind sie schneller]",[1],{GK},{Rückgabe an Sichter}
00f52db8-86e3-46c9-b9d1-36361fd812cc,"[Shock Therapy Achtung ! Wegen Portierungsproblemen zwischen der Telekom und Vodafone sind wir telefonisch nicht erreichbar ! Raus telefonieren können wir aber . Schickt uns eine e-mail dann rufen wir zurück oder versucht es unter 0176-0123456 7890 Alternativ könnt ihr auch unser Kontaktformular auf der Website verwenden . http://shock-therapy.com/ueber-shock-therapy/kontaktformular/ Das ist einfach eine riesen Sauerei die da läuft ! Vodafone Deutschland schiebt es auf die Deutsche Telekom AG und die Telekom sagt lapidar man müsse sich an den neuen Anbieter wenden . Soweit "" In Deutschland ist alles geregelt ! ! Shock Therapy shared their photo . https://scontent.xx.fbcdn.net/hphotos-xfp1/v/t1.0-0/s130x130/01234567890_0123456789001234567890_0123456789001234567890_n.jpg?oh=b32af0557c01234567890ce9de9f24d992ade&oe=57BF609D]",[1],{Wechsler},{nan}
01503e4d-59e7-4754-9408-b0569cc8b20f,"[Hallo , umzugsbedingt haben wir ein Sonderkündigungsrecht der Telekom genutzt , was aber wohl durch die Rufnummerübernahme zum neuen Anbieter mittlerweile seit November 2015 nur zu Chaos führt . Ständig bekommen wir neue Schreiben , das unser Vertrag mit der Telekom nun gekündigt sei , dann kommt kurz darauf ein Schreiben was besagt , dass unser Vertrag weiter besteht , usw , usw . Bei der Hotline blickt auch niemand mehr durch - oftmals ist das Gespräch "" einfach weg "" ! ! ! Sorry , aber diese Callcenter sind einfach nur Kundenfeindlich ! Muss ich bis nach Bonn fahren , um jmd kompetentes zu erwischen , der endlich helfen kann ? ? ?]",[1],{Wechsler},{nan}
028dae36-ed94-40f0-8a80-cdf2af1a97ba,"[Liebes Telekom-hilft-Team , mein Vorgang 01234567890 #01234567890 # flammt wieder auf : Mein Festnetz-Anschluss ist von außen nicht erreichbar , weder mobil aus einem anderen Netz ( "" Anschluss vorübergehend nicht erreichbar "" ) noch aus anderen Festnetzen . Der Versuch , die an der ISDN-Anlage angeschlossenen Festnetznummern gegenseitig anzurufen , endet in einem Besetztzeichen . Kurz : Der Zustand , den wir schon einmal verbessert hatten , hat sich wieder verschlechtert , bei ungeänderter Konfiguration der Anlage und des Routers . Was tun ? Hier scheint das Routing – wieder einmal – erheblich durcheinander gekommen zu sein . Über ein Rückmeldung freue ich mich – ist einfach nicht so schön , wenn die Kunden erst ins Leere laufen und dann mobil anrufen und das erste Thema im Gespräch die eigene Nichterreichbarkeit ist ...]",[1],{GK},{nan}
031ff757-3678-4c9e-a8fb-06c354a2ea2c,"[@Telekom_hilft Und warum nur Magenta ? Ich hätte auch gern mehr mit DB Flex ., @Telekom_hilft Und warum nur Magenta ? Ich hätte auch gern mehr mit DB Flex .]","[1, 1]",{GK},{nan}
04892a95-2e6c-4a8d-9453-408283a3b5db,[@Telekom_hilft Seid Ihr auch für Geschäfsk . zuständig ? Die Kollegen per Mail legen scheinbar nicht so viel Wert auf 15 + Jahre Kundschaft .],[1],{GK},{nan}
055a31f2-ac05-4876-b0da-d4dedd63c170,"[Guten Abend , obw

In [78]:
import re
GRUBER_URLINTEXT_PAT = re.compile(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?\xab\xbb\u201c\u201d\u2018\u2019]))')
NUMBER_CLEAN = re.compile(r'\d+(:?[.,]?\d+)*')
TIME_CLEAN = re.compile(r'\d+:\d+(?:\s?[u]hr)?')
PUNCT_CLEAN = re.compile(r'(\s?)[\.\,()\[\]\{\}\^\:\'\"\/;/!/?/…/€/%//“„=–><&]+(\s?)')
DASH_CLEAN = re.compile(r'([^et])[-]')
TWEET_CLEAN = re.compile(r'@\w+')
HASH_CLEAN = re.compile(r'#(\w+)')
SPACE_CLEAN = re.compile(r'(\s)\s+')
RT_CLEAN = re.compile(r'^rt\b')

def preprocessing(text):
    try:
        text = text.lower()
        # clean-up url
        text = GRUBER_URLINTEXT_PAT.sub('', text)
        # remove time
        text = TIME_CLEAN.sub('', text)
        # clean-up numbers
        text = NUMBER_CLEAN.sub('', text)
        text = PUNCT_CLEAN.sub(r'\1\2',text)
        text = TWEET_CLEAN.sub('', text)
        text = HASH_CLEAN.sub(r'\1', text)
#         text = DASH_CLEAN.sub(r'\1 ', text)
        text = SPACE_CLEAN.sub(r'\1', text)
        text = RT_CLEAN.sub(r'', text)
        return ' '.join([word for word in text.split(' ') if word not in stopwords]).strip()
    except:
        return ''

with open('./stopwords-de.txt') as f:
    stopwords = f.read().split('\n')

In [79]:
documents = [] # Content
documents_raw = []
labels = [] # SichterName

for (idx, row) in df.iterrows():
    if not row.Content:
        continue
    labels.append(row.SichterName)
    documents_raw.append(row.Content)
    documents.append(' '.join([preprocessing(text) for text in documents_raw[-1] if text]))

In [80]:
documents[:10]

['liebe telefonanschluss aktiviert schön scheint schwierig zeitnah eigentliche kerngeschäft kümmern mitarbeiter liebe kunden geschäft mangels fachkompetenz leider telefon fax deutschland afrika tiefsten busch leben verständnis buschtrommel benutzen rauchzeichen unserem ort aufgehen freundliche grüße eva-maria gebels',
 'weitergeholfen bekommen glaube interessiert verein schade bezahlt rechnung schneller',
 'shock therapy achtung portierungsproblemen telefonisch erreichbar raus telefonieren schickt e-mail rufen versucht - alternativ kontaktformular website verwenden einfach riesen sauerei läuft deutschland schiebt deutsche lapidar müsse anbieter wenden deutschland geregelt shock therapy shared their photo',
 'umzugsbedingt sonderkündigungsrecht genutzt rufnummerübernahme anbieter mittlerweile chaos führt ständig bekommen schreiben vertrag gekündigt schreiben besagt vertrag besteht usw usw hotline blickt - oftmals gespräch einfach sorry callcenter einfach kundenfeindlich bonn fahren jmd 

In [81]:
from gensim import corpora, models

dictionary = corpora.Dictionary([document.split(' ') for document in documents])

In [82]:
corpus = [dictionary.doc2bow(document.split(' ')) for document in documents]

In [83]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=N_TOPICS, id2word=dictionary, passes=N_PASSES)

In [84]:
topics_list = dict(ldamodel.print_topics(num_topics=30, num_words=4))

In [85]:
topics_list

{0: '0.016*"adresse" + 0.015*"absage" + 0.013*"-" + 0.010*"internet"',
 1: '0.025*"techniker" + 0.014*"anruf" + 0.008*"-" + 0.008*"telefon"',
 2: '0.008*"surfen" + 0.008*"deutschland" + 0.008*"shock" + 0.008*"therapy"',
 3: '0.012*"techniker" + 0.010*"-" + 0.010*"telefon" + 0.010*"hilfe"',
 4: '0.008*"magenta" + 0.008*"echt" + 0.005*"router" + 0.005*"erreichbar"',
 5: '0.010*"vertrag" + 0.008*"neues" + 0.006*"kunden" + 0.006*"nix"',
 6: '0.019*"hotline" + 0.012*"isdn" + 0.009*"teilte" + 0.008*"mitarbeiter"',
 7: '0.017*"hotline" + 0.012*"vertrag" + 0.010*"kündigen" + 0.007*"laut"',
 8: '0.012*"techniker" + 0.011*"anschluss" + 0.010*"-" + 0.010*"leider"',
 9: '0.027*"-" + 0.013*"techniker" + 0.011*"internet" + 0.011*"unserem"',
 10: '0.012*"-" + 0.008*"dtag" + 0.008*"kunden" + 0.006*"einfach"',
 11: '0.012*"internet" + 0.011*"-" + 0.011*"voip" + 0.008*"technik"',
 12: '0.007*"telefon" + 0.007*"hotline" + 0.007*"termin" + 0.007*"business"',
 13: '0.019*"-" + 0.008*"hotline" + 0.008*"busi

In [86]:
a = ldamodel[dictionary.doc2bow(documents[10].split(' '))]

In [87]:
documents[10]

'fühle stich gelassen schriftliche auftragsbestätigung techniker termine techniker offline verärgert'

In [88]:
sorted(a, key=lambda x: x[1],reverse=True)[0][0], topics_list[10]

(1, '0.012*"-" + 0.008*"dtag" + 0.008*"kunden" + 0.006*"einfach"')

# GET MORE LABELS FROM TELEKOM.DE/HILFE

In [89]:
from itertools import product
import pickle
items = pickle.load(open('./cleaned_answers_2.pickle','rb'))

qa = []
qa_labels = []

for item in items:
    KQ = [kq.decode('utf-8') for kq in item['key_questions']]
    answer = item['answer'].decode('utf-8')
    question = item['question'].decode('utf-8')
    if KQ:
        combs = list(product([question], KQ, [answer]))
    else:
        combs = list(product([question], [answer]))
    qa_labels += [item['link'].split('>')] * len(combs)
    qa += [' '.join(comb) for comb in combs]

In [90]:
# bag of words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

BoW = CountVectorizer(documents+qa, stop_words=stopwords, strip_accents='unicode', ngram_range=(1,3), min_df=3)
X_bag = BoW.fit_transform(documents+qa)
tf_transformer = TfidfTransformer()
X_bag = tf_transformer.fit_transform(X_bag)

In [91]:
vectors = []
clusters = []
for document in documents:
    cluster = ldamodel[dictionary.doc2bow(document.split(' '))]
    clusters.append(sorted(a, key=lambda x: x[1],reverse=True)[0][0])
    vector = BoW.transform([document])
    vector = tf_transformer.transform(vector)
    vectors.append(vector)
    
qa_vectors = []
for document in qa:
    vector = BoW.transform([document])
    vector = tf_transformer.transform(vector)
    qa_vectors.append(vector)

In [92]:
from sklearn.metrics.pairwise import linear_kernel
from scipy.sparse import vstack

vectors = vstack(vectors)
qa_vectors = vstack(qa_vectors)

sim = linear_kernel(vectors, qa_vectors)
sim.shape

(301, 1655)

In [93]:
def get_new_labels(document):
    print(document)
    topic = topics[documents == document]
    print(topics_list)

In [94]:
def get_topic(document):
    topics = ldamodel[dictionary.doc2bow(document.split(' '))]
    topic_ind = sorted(topics, key=lambda x: x[1],reverse=True)[0][0]
    return topic_ind, ldamodel.print_topics(num_topics=30, num_words=10)[topic_ind]

In [95]:
sort_sim = sim.argsort(axis=0)
sort_sim.shape

(301, 1655)

In [96]:
def get_qa(document_ind):
    sort_sim[-1:document_ind]

In [97]:
get_topic(documents[7135])

IndexError: list index out of range

In [ ]:
topics = [get_topic(document)[0] for document in documents]
topics = np.array(topics)

In [ ]:
from collections import defaultdict
new_labels = defaultdict(list)
new_topics = defaultdict(set)

k = 4
l = 2
granularity = 2
best_sort = sort_sim[-k:,:]

for i in range(len(qa_labels)):
    if len(np.unique(topics[best_sort[:,i]])) <= l:
        best_topic = np.bincount(topics[best_sort[:,i]]).argmax()
        new_topics[topics[best_topic]] |= (set(qa_labels[i][:granularity]))

In [ ]:
new_topics, topics_list

In [ ]:
topics[[30864, 25575, 17414, 33209]]

In [ ]:
len(qa_labels)

In [ ]:
new_topics

In [ ]:
documents[283]

In [ ]:
new_labels

In [ ]:
len(qa_labels)

In [ ]:
len(topics)

In [ ]:
documents[7135], qa_labels[1403], qa[1403]